In [ ]:
!pip install langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade langchain langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain-exa langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_groq

In [ ]:
%pip install -qU duckduckgo-search langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.0 MB/s eta 0:00:00


In [ ]:
from typing import List
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent


In [ ]:
EXA_API_KEY=""
GROQ_API_KEY=""

In [ ]:
from typing_extensions import TypedDict
class CoverLetterMemory(TypedDict):
  name:str
  email:str
  phone:str
  skills:list[str]
  projects:list[str]
  experience:list[str]
  job_description: str
  cover_letter: str



In [ ]:
def collect_user_input() -> CoverLetterMemory:
    name = input("Enter your full name: ")
    email = input("Enter your email address: ")
    phone = input("Enter your phone number: ")

    skills = input("Enter your skills (comma separated): ").split(",")
    projects = input("Enter your projects (comma separated): ").split(",")
    experience = input("Enter your experiences (comma separated): ").split(",")

    skills = [s.strip() for s in skills]
    projects = [p.strip() for p in projects]
    experience = [e.strip() for e in experience]

    return {
        "name": name,
        "email": email,
        "phone": phone,
        "skills": skills,
        "projects": projects,
        "experience": experience
    }

In [ ]:
user_temperature=1

In [ ]:
from langchain_groq import ChatGroq

groq_llm=ChatGroq(
    api_key=GROQ_API_KEY,
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=user_temperature,
)

In [ ]:
groq_llm.invoke("how are you today").content

"I'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How about you? How's your day going?"

In [ ]:
job_description="""Job Title: Entry-Level AI Engineer
Location: Remote or Hybrid (San Francisco, CA)
Employment Type: Full-time

About the Role:
We are seeking a passionate and highly motivated Entry-Level AI Engineer to join our fast-growing AI research and product development team. The ideal candidate should have a strong foundation in machine learning, deep learning frameworks, and programming in Python.

Key Responsibilities:

Assist in designing and developing AI models and pipelines.

Collaborate with cross-functional teams to integrate AI solutions into products.

Conduct research on emerging AI techniques and tools.

Help in evaluating model performance and optimizing algorithms.

Write clean, maintainable, and well-documented code.

Requirements:

Bachelor's degree in Computer Science, Artificial Intelligence, Data Science, or related field.

Strong understanding of machine learning algorithms and Python programming.

Familiarity with TensorFlow, PyTorch, or similar deep learning frameworks.

Good problem-solving and communication skills.

Internship or academic project experience in AI/ML is a plus.

Preferred Qualifications:

Knowledge of NLP, computer vision, or reinforcement learning.

Familiarity with cloud platforms (AWS, GCP) and model deployment.

Open-source contributions or participation in AI hackathons.

Why Join Us?

Work with a collaborative and innovative team.

Opportunity to grow and shape cutting-edge AI products.

Competitive salary, flexible work culture, and professional development support."""

In [ ]:
def cover_letter(memory: CoverLetterMemory):
    '''Generate a cover letter from the job description and applicant info.'''
    prompt = f"""
You are an AI assistant that writes tailored cover letters based on job descriptions and applicant details.

Job Description:
{memory["job_description"]}

Applicant Information:
Name: {memory['name']}
Email: {memory['email']}
Phone: {memory['phone']}
Skills: {", ".join(memory['skills'])}
Projects: {", ".join(memory['projects'])}
Experience: {", ".join(memory['experience'])}

Write a professional and personalized cover letter.
"""
    memory["cover_letter"]=groq_llm.invoke(prompt)
    return memory["cover_letter"]


In [ ]:
def search_web(query:str):
  '''Search the web using exatool'''
  search=ExaSearchRetriever(k=3, highlights=True, exa_api_key=EXA_API_KEY)
  return search.invoke(query)

result=search_web("what is the capital of India")

for doc in result:
    print("\n--- Main Content ---\n")
    trimed_res=doc.page_content[:500]
    print(trimed_res)
    print("\n--------------------\n")


--- Main Content ---

This article is about the capital of India, within the union territory of [Delhi](https://en.wikipedia.org/wiki/Delhi). For other uses, see [New Delhi (disambiguation)](https://en.wikipedia.org/wiki/New_Delhi_(disambiguation)).

| New Delhi _Naī Dillī_ |
| --- |
| [Federal capital](https://en.wikipedia.org/wiki/Federal_capital) |
| [Rashtrapati Bhavan](https://en.wikipedia.org/wiki/Rashtrapati_Bhavan) [Parliament House](https://en.wikipedia.org/wiki/Parliament_House,_New_Delhi) [Bharat Mandapam]

--------------------


--- Main Content ---

- [GK for competitive exams](https://www.geeksforgeeks.org/gk-general-knowledge-questions/?ref=shm)
- [General Awareness for Bank Exams](https://www.geeksforgeeks.org/general-awareness-for-bank-exam/?ref=shm)
- [Current Affairs](https://www.geeksforgeeks.org/current-affairs/?ref=shm)
- [Govt Schemes](https://www.geeksforgeeks.org/list-of-important-government-schemes-india-2023/?ref=shm)
- [Financial Awareness](https://www.geek

In [ ]:
cover_letter_prompt = """
You are an intelligent AI assistant that helps users generate personalized, professional cover letters for job applications.

You have access to a tool called `cover_letter`, which takes a job description and user profile data (like name, skills, experience) and returns a complete cover letter.

When a user provides a job description or asks you to generate a cover letter, use the `cover_letter` tool with the appropriate inputs.

Make sure the output is professional, well-formatted, and tailored to the role described.

If the user hasn’t provided their details yet, ask them for the following before calling the tool:
- Full Name
- Email
- Phone
- Education
- Skills (comma-separated)
- Projects (comma-separated)
- Experience (comma-separated)
"""


In [ ]:
cover_letter_agent=create_react_agent(
    model=groq_llm,
    tools=[cover_letter],
    name="cover_letter_agent",
    prompt=cover_letter_prompt,
)

In [ ]:
result=cover_letter(job_description,user_data)
print(result)

content="Here's a tailored cover letter based on the job description and applicant details:\n\nDear Hiring Manager,\n\nI am excited to apply for the Entry-Level AI Engineer position at [Company Name]. As a highly motivated and passionate individual with a strong foundation in machine learning and programming, I am confident that I would be an excellent fit for your fast-growing AI research and product development team.\n\nWith a [Degree] in [Field of Study] from [University Name], I possess a solid understanding of machine learning algorithms, deep learning frameworks, and Python programming. My academic background has provided me with a strong foundation in computer science, and I am eager to apply my skills and knowledge to real-world problems. I am particularly drawn to [Company Name] because of its commitment to innovation and cutting-edge AI products.\n\nAs a detail-oriented and organized individual, I am well-equipped to assist in designing and developing AI models and pipelines,

In [ ]:
def collect_user_info_node(state:dict)->dict:
  user_info=collect_user_input()
  state.update(user_info)
  return state

In [ ]:
def collect_job_disc(state:dict)->dict:
  job_disc=input("enter the job disc")
  state["job_description"]=job_disc
  return state

In [ ]:
from langgraph.graph import StateGraph

builder=StateGraph(CoverLetterMemory)

builder.add_node("collect_user_info",collect_user_info_node)
builder.add_node("collect_job_dic",collect_job_disc)
builder.add_node("cover_letter_agent",cover_letter_agent)


builder.set_entry_point("collect_user_info")
builder.add_edge("collect_user_info","collect_job_dic")
builder.add_edge("collect_job_dic","cover_letter_agent")

builder.set_finish_point("cover_letter_agent")

graph=builder.compile()

In [ ]:
result=graph.invoke({})
print(result)

Enter your full name: a
Enter your email address: a
Enter your phone number: a
Enter your skills (comma separated): a
Enter your projects (comma separated): a
Enter your experiences (comma separated): a
enter the job discai engineer


BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: parameters for tool cover_letter did not match schema: errors: [missing properties: 'job_description', 'memory']", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'I understand the guidelines. I will follow them to provide helpful responses. \n\nTo confirm, I have access to the following function:\n\n[\n {"name":"cover_letter","description":"Generate a cover letter from the job description and applicant info.","parameters":{"properties":{"job_description":{"type":"string"},"memory":{"properties":{"email":{"type":"string"},"experience":{"items":{"type":"string"},"type":"array"},"name":{"type":"string"},"phone":{"type":"string"},"projects":{"items":{"type":"string"},"type":"array"},"skills":{"items":{"type":"string"},"type":"array"}},"required":["name","email","phone","skills","projects","experience"],"type":"object"}},"required":["job_description","memory"],"type":"obj